In [ ]:
import pandas as pd
import keras.utils as image
from tensorflow import keras
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
import cv2
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('dataset_processed-Copy/df.csv')

In [ ]:
df.head()

In [ ]:
def compute_optical_flow(df):
    flow_data = []
    for i, row in df.iterrows():
        # Check if the next frame is from the same video
        next_row = df.iloc[i + 1] if i + 1 < len(df) else None
        if next_row is not None and row['video_name'] != next_row['video_name']:
            continue

        # Read the current frame and the next frame
        frame1 = cv2.imread(row['file_path'])
        if next_row is not None:
            frame2 = cv2.imread(next_row['file_path'])
        else:
            # If the next frame is from a different video, skip computing optical flow
            continue
        
        frame1 = cv2.resize(frame1, (224, 224))
        frame2 = cv2.resize(frame2, (224, 224))

        # Convert the frames to grayscale
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Compute the optical flow between the frames
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Convert flow vectors to polar coordinates
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Threshold the magnitude to create a binary mask of moving regions
        magnitude_threshold = 4
        magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        ret, mask = cv2.threshold(magnitude, magnitude_threshold, 1, cv2.THRESH_BINARY)

        # Apply the mask to the second frame to highlight the moving regions
        frame2_masked = cv2.bitwise_and(frame2, frame2, mask=mask)

        # Subtract the masked second frame from the first frame to create a difference image
        diff = cv2.absdiff(frame1, frame2_masked)

        # Convert difference image to colored
        diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

        # Store the optical flow data for the current pair of frames
        flow_data.append({
            'video_name': row['video_name'],
            'frame_name': row['frame_name'],
            'moition_residual': diff,
            'label': row['label'],
            'split': row['split']
        })
    return pd.DataFrame(flow_data)

df = pd.read_csv('dataset_processed-Copy/df.csv')
flow_df = compute_optical_flow(df)
flow_df.to_csv('dataset_processed-Copy/flow_df.csv')

# flow_df.head()